### Sometimes we're interested in accessing reanalysis data relative to its local time as opposed to UTC (the grid it's usually produced on). For example, CloudSat-CALIPSO observations occur at 1:30 AM & 1:30 PM local time for scenes. Say we want to co-locate these observations with models or understand how this local time sampling may affect the PDFs of a variable's distribution. We must convert reanalysis data into local-time space. This essentially involves cutting and pasting grids together as a function longituide at different model reanalysis time. Luckily we can do this pretty efficientlly with xarray.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import glob
import cartopy
import cartopy.crs as ccrs
import matplotlib.colors as mplc
import matplotlib.patches as mpatches
import datetime

def moving_average(a, n=2) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

h2s = 1/3600. #hours to seconds conversion

In [ ]:
#point towards data path
path = '/boltzmann/data1/Reanalyses/ERA5/polar_hourly/*ARC*'
files = np.sort(glob.glob(path))

#open ERA5 data
ds = xr.open_mfdataset(files)
ds = ds.reindex(latitude=ds.latitude[::-1]).sel(latitude=slice(60,82)) #reverse order of latitude and take NH polar slice

#access longwave flux towards surface
ds['flds'] = ds['strd'] / h2s #convert accumulated value to instantaneous
flds = ds.flds

In [ ]:
# Define the longitude-based time offset--
# Since the Earth rotates 360 degrees in 24 hours, each hour corresponds to 15 degrees of longitude
# Therefore, we divide each longitude value by 15 to convert it from degrees to hours,
# representing the time offset from the Prime Meridian. For example, 
# longitude of   90 E corresponds to 6 hours  ahead the PM (UTC+6)
# longitude of -120 E corresponds to 8 hours behind the PM (UTC-8)
time_offset = (ds['longitude'] / 15 * h2s).astype("timedelta64[s]")

# Calculate local time
local_time = ds['time'] + time_offset

# Round local time to the nearest hour
local_time_rounded = local_time.dt.round('H')

# Mask to select specific hours (1 AM and 1 PM), this aligns with CS-C sampling times
time_mask = ((local_time_rounded.dt.hour == 1) | (local_time_rounded.dt.hour == 13))

# Select data based on the mask
flds_sub = flds.where(time_mask)